# Demonstration of MCMC non-linear regression with EMCEE and refnx

`refnx` is a package that can be used for non-linear regression (curvefitting). Here I demonstrate how it can be used to analyse Gaussian curve dataset, with Bayesian MCMC sampling of the posterior distributions of the parameters. This is a very robust way of estimating parameter uncertainties. I will also do the analysis with the `emcee` package for comparison

The first step is all the imports.

In [ ]:
import numpy as np
import emcee
import corner
from scipy.optimize import leastsq
from refnx.analysis import CurveFitter, Parameter, Parameters, Model, Objective
from refnx.dataset import Data1D
import matplotlib.pyplot as plt
%matplotlib inline

First step is to load some data in.

In [ ]:
data = Data1D('gauss_data.txt')
plt.errorbar(data.x, data.y, yerr=data.y_err, fmt='.k')

Define the fit function.

In [ ]:
def gauss(x, p, *args):
    # p is a Parameters instance. A quick way of getting all the numerical values out
    # is making it into array. However, there alternate ways of access:
    # e.g. p['bkg'].value or p[0].value.
    p0 = np.array(p)
    return p0[0] + p0[1] * np.exp(-((x - p0[2]) / p0[3])**2)

Set up initial parameter guesses and lower and upper bounds. The last step is to create a `refnx.Parameters` instance.

In [ ]:
bkg = Parameter(0.1, 'bkg', vary=True, bounds=(-1, 1))
amp = Parameter(20, 'amp', vary=True, bounds=(0, 30))
mu = Parameter(0.1, 'mu', vary=True, bounds=(-5, 5))
wid = Parameter(0.1, 'wid', vary=True, bounds=(0.001, 2))

# to get numerical values out of p0 you have to use np.array(p0), or refer to each Parameter
# by using p0['bkg'].value or p0[0].value.
p0 = bkg | amp | mu | wid

## Analyse with emcee

To start with we'll do the analysis with the `emcee` package. Then we'll repeat the analysis with `refnx.analysis.CurveFitter`. 

The following functions have to be defined for `emcee`. The log-likelihood, the uniform log-prior and the overall log-posterior probability.

In [ ]:
bounds_varying = np.array([[-1, 0, -5, 0.001], [1, 30, 5, 2]]).T

def residuals(theta):
    resid = (gauss(data.x, theta) - data.y) / data.y_err
    return resid
    
def lnlike(theta):
    # log likelihood
    return -0.5 * (np.sum(residuals(theta) ** 2))

def lnprior(theta):
    # uniform prior
    if (np.any(theta > bounds_varying[:, 1])
            or np.any(theta < bounds_varying[:, 0])):
        return -np.inf
    return 0

def lnpost(theta):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta)

Lets fit the data with least squares first.

In [ ]:
result = leastsq(residuals, p0, full_output=True)
best_fit = result[0]
best_errors = np.sqrt(np.diag(result[1]))
for mean, std in zip(best_fit, best_errors):
    print("{:<12g} +/-  {:<10g}".format(mean, std))

Set up the walkers for `emcee`.

In [ ]:
ndim, nwalkers = 4, 100
pos = np.array([np.array(p0) * (1 + 1e-2 * np.random.randn(ndim))
                for i in range(nwalkers)])

Run the `emcee` sampler

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnpost)
a = sampler.run_mcmc(pos, 1000)

Discard 100 burn in steps for each walker and flatten the chain.

In [ ]:
chain = sampler.chain[:, 100:, :].reshape(-1, 4)

## Analyse with CurveFitter

Now we're going to do the analysis using a `refnx.analysis.CurveFitter` instance, it should be a lot simpler than the direct approach above. First setup the curvefitter.

In [ ]:
# first setup a model
model = Model(p0, fitfunc=gauss)

# an objective is composed of a model and data
objective = Objective(model, data)

# a fitter is constructed
fitter = CurveFitter(objective, nwalkers=100, threads=4)

First of all do a least-squares fit, to get a starting point for the sampling.

In [ ]:
res_leastsq = fitter.fit('least_squares')

Now do the MCMC sampling with CurveFitter instead. There are 100 walkers, we do 1000 steps on each walker. After the sampling discard the first 100 steps of each walker and take every 5th step

In [ ]:
fitter.sample(1000)
res_sampling = fitter.process_chain(nburn=100, nthin=5)

The following plot shows the posterior distributions for each parameter

In [ ]:
b = corner.corner(fitter.sampler.flatchain)

But what about the fits, are they good?

In [ ]:
plt.errorbar(data.x, data.y, yerr=data.y_err, fmt=".")

saved_state = np.array(p0)
# plot a selection of the samples
for pars in objective.pgen(500):
    # could also use:
    # >>> objective.setp(pars)
    # then to calculate the model:
    # >>> model(data.x)
    plt.plot(data.x, objective.generative(pars), color="k", alpha=0.02)

plt.plot(data.x, gauss(data.x, p0), color='r', label='sampling')
objective.setp(saved_state)

# the leastsq fit overlies the sampling
# plt.plot(data.x, gauss(data.x, best_fit), color='g', label='leastsq')
plt.legend()

The following fit parameters are obtained. Lets compare them to the least squares output.

In [ ]:
print("Curvefitter.sampling")
print(objective)

print("\nleastsq")
print("-------")
print(best_fit, '\n', best_errors)